# Concurrency in Python

### GIL
In Python's main interpreter implementation, the one we all use (CPython), there is a __GIL__ or Global Interpreter Lock that ensure only one thread may execute Python bytecode at a time. This is because CPython is not thread safe.

As such, we really need to look at multiprocessing when we're trying to use multiple cores effectively in pure Python.

### Data structures and pickle
To communicate between processes, both concurrent.futures (the main implementation of multiprocessing and multiprocessing in the Standard Library) and other implementation rely on pickle, which allows saving Python objects into files.

However, not all objects can be _'pickled'_. Namely, you can't pickle a lot of things including:
- Lambdas
- Default dictionaries (defaultdict)
- Custom objects

Please refer to [Pickle's documentation](https://docs.python.org/3/library/pickle.html#what-can-be-pickled-and-unpickled) for more information on what can and can't be pickled.

## Single-threaded

In [ ]:
# Setup
import random
COUNT = pow(10, 6)
random_values = {random.random(): [] for _ in range(100)}

In [ ]:
# Execution
for random_value, inner_values in random_values.items():
    for i in range(COUNT):
        inner_values.append(random_value * random.random())

In [ ]:
import gc
del random_values
gc.collect()

## Multiprocess

### Standard Library

In [ ]:
# Setup
random_values_parallel = {random.random() for _ in range(100)}

def inner_loop(random_value):
    inner_values = []
    for i in range(COUNT):
        inner_values.append(random_value * random.random())
    return random_value,  inner_values

from concurrent.futures import ProcessPoolExecutor
pool = ProcessPoolExecutor()

In [ ]:
random_values_parallel_filled = dict(pool.map(inner_loop, random_values_parallel))

In [ ]:
del random_values_parallel_filled
gc.collect()

### Loky

In [ ]:
# Setup
from loky import get_reusable_executor
pool = get_reusable_executor()

In [ ]:
random_values_parallel_filled = dict(pool.map(inner_loop, random_values_parallel))

In [ ]:
del random_values_parallel_filled
gc.collect()